#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
##PEC4: Segmentación comportamental basada en clustering (ML no supervisado) 
## UOC - Máster en Data Science
### Alumno: **Fernando Antonio Barbeiro Campos** - fbarbeiro@uoc.edu

Esta PEC simula el ejercicio completo de generación de segmentos (clusters) de clientes utilizando un dataset de usuarios y evaluaciones de películas liberados por una empresa de streaming multimedia bajo demanda por Internet.

**Este notebook cubre:**
* *Parte 1:* ETL de los ficheros (1 puntos)
* *Parte 2:* Ingeniería de características (2 puntos)
* *Parte 3:* Clustering (2 puntos)
* *Parte 4:* Validación de los resultados (2.5 puntos)
* *Parte 5:* Normalización de los datos (2.5 puntos)

**IMPORTANTE:** Versión de spark para realizar esta PAC: 2.3

## Parte 1: ETL de los ficheros

** Resumen **

Los archivos que usaremos contienen 1.000.209 calificaciones anónimas de aproximadamente 3.900 películas de 6.040 usuarios realizadas durante el año 2000.

** Fichero de calificaciones **

Todas las calificaciones están contenidas en el archivo "ratings.dat" y tienen el siguiente formato:

*UserID::MovieID::Rating::Timestamp*

- UserIDs: cuyo rango se encuentra entre 1 y 6040
- MovieIDs: cuyo rango se encuentra entre 1 y 3952
- Rating: las calificaciones se realizan en una escala de 5 estrellas (sin decimales)
- Timestamp: la marca de tiempo se representa en segundos

Nota: Para cada usuario hay como mínimo 20 calificaciones

** Fichero de usuarios **

La información sobre los usuarios está contenida en el fichero "users.dat" y tiene el siguiente formato:

*UserID::Gender::Age::Occupation::Zip-code*

Toda la información demográfica es proporcionada voluntariamente por los usuarios y no se comprueba su exactitud. Solo los usuarios que hayan proporcionado datos demográficos se incluyen en este conjunto de datos.

- Gender: El género se denota por "M" para hombres y "F" para mujeres
- Age: Distibuida en los siguientes rangos:

	*  1:  "- 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56 - "

- Occupation: Se elige de las siguientes opciones:

	*  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
    
** Fichero de películas **

La información se ecuentra en el archivo "movies.dat" y está en el siguiente
formato:

*MovieID::Title::Genres*

- Title: Los títulos son idénticos a los títulos proporcionados por la IMDB (incluyendo año de lanzamiento)
- Genres: Los géneros están separados por *pipes* y se seleccionan de la siguiente lista:
    * Action
	* Adventure
	* Animation
	* Children's
	* Comedy
	* Crime
	* Documentary
	* Drama
	* Fantasy
	* Film-Noir
	* Horror
	* Musical
	* Mystery
	* Romance
	* Sci-Fi
	* Thriller
	* War
	* Western
    
- Algunos MovieIDs no corresponden a una pelicula debido a un duplicado accidental a su entradas y/o a entradas de prueba
- Las películas se introducen principalmente a mano, por lo que pueden existir errores e incoherencias

### Ejercicio 1(a):

Empezaremos por visualizar una muestra de los datos. Para esto usaremos las funciones pre-definidas en los notebooks de Databricks para explorar su sistema de archivos.

Usar `display(dbutils.fs.ls("/databricks-datasets/cs110x/ml-1m/data-001")` para mostrar la descripción de los datos que se van a usar.

In [4]:
#TODO: use display to list all the files of the directory containing the data
dbfs_dir = '/databricks-datasets/cs110x/ml-1m/data-001'
display(dbutils.fs.ls(dbfs_dir))

path,name,size
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/README,README,5577
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/movies.dat,movies.dat,171308
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/movies.dat.gz,movies.dat.gz,58049
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/ratings.dat,ratings.dat,24594131
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/ratings.dat.gz,ratings.dat.gz,5816216
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/users.dat,users.dat,134368
dbfs:/databricks-datasets/cs110x/ml-1m/data-001/users.dat.gz,users.dat.gz,40690


Ahora usaremos el comando `print dbutils.fs.head` para visualizar el contenido de los ficheros "movies.dat", "user.dat" y "ratings.dat"

In [6]:
#TODO: use dbutils.fs.head to inspect the file "user.dat"
users_filename = dbfs_dir + '/users.dat' 
print(dbutils.fs.head(users_filename))

[Truncated to first 65536 bytes]
1::F::1::10::48067
2::M::56::16::70072
3::M::25::15::55117
4::M::45::7::02460
5::M::25::20::55455
6::F::50::9::55117
7::M::35::1::06810
8::M::25::12::11413
9::M::25::17::61614
10::F::35::1::95370
11::F::25::1::04093
12::M::25::12::32793
13::M::45::1::93304
14::M::35::0::60126
15::M::25::7::22903
16::F::35::0::20670
17::M::50::1::95350
18::F::18::3::95825
19::M::1::10::48073
20::M::25::14::55113
21::M::18::16::99353
22::M::18::15::53706
23::M::35::0::90049
24::F::25::7::10023
25::M::18::4::01609
26::M::25::7::23112
27::M::25::11::19130
28::F::25::1::14607
29::M::35::7::33407
30::F::35::7::19143
31::M::56::7::06840
32::F::25::0::19355
33::M::45::3::55421
34::F::18::0::02135
35::M::45::1::02482
36::M::25::3::94123
37::F::25::9::66212
38::F::18::4::02215
39::M::18::4::61820
40::M::45::0::10543
41::F::18::4::15116
42::M::25::8::24502
43::M::25::12::60614
44::M::45::17::98052
45::F::45::16::94110
46::M::18::19::75602
47::M::18::4::94305
48::M::25::4::92107
49::M::18::12::77084
50::F::25::2::98133
51::F::1::10::10562
52::M::18::4::72212
53::M::25::0::96931
54::M::50::1::56723
55::F::35::12::55303
56::M::35::20::60440
57::M::18::19::30350
58::M::25::2::30303
59::F::50::1::55413
60::M::50::1::72118
61::M::25::17::95122
62::F::35::3::98105
63::M::18::4::54902
64::M::18::1::53706
65::M::35::12::55803
66::M::25::18::57706
67::F::50::5::60181
68::M::18::4::53706
69::F::25::1::02143
70::M::18::4::53703
71::M::25::14::95008
72::F::45::0::55122
73::M::18::4::53706
74::M::35::14::94530
75::F::1::10::01748
76::M::35::7::55413
77::M::18::4::15321
78::F::45::1::98029
79::F::45::0::98103
80::M::56::1::49327
81::F::25::0::60640
82::M::25::17::48380
83::F::25::2::94609
84::M::18::4::53140
85::M::18::4::94945
86::F::1::10::54467
87::M::25::14::48360
88::F::45::1::02476
89::F::56::9::85749
90::M::56::13::85749
91::M::35::7::07650
92::F::18::4::44243
93::M::25::17::95825
94::M::25::17::28601
95::M::45::0::98201
96::F::25::16::78028
97::F::35::3::66210
98::F::35::7::33547
99::F::1::10::19390
100::M::35::17::95401
101::F::18::3::33314
102::M::35::19::20871
103::M::45::7::92104
104::M::25::12::00926
105::M::45::12::90277
106::F::35::11::79101
107::M::45::18::63129
108::M::25::12::30316
109::M::45::15::92028
110::M::25::2::90803
111::M::35::15::55416
112::M::25::16::97209
113::M::18::12::37032
114::F::25::2::83712
115::M::25::17::28083
116::M::25::17::55744
117::M::25::17::33314
118::M::35::17::22315
119::F::1::10::77515
120::M::25::11::27106
121::M::35::7::75229
122::F::18::4::94305
123::M::35::9::67208
124::M::56::7::91356
125::M::45::14::01701
126::M::18::9::98117
127::F::45::3::01770
128::M::56::6::37922
129::M::25::11::20164
130::M::35::17::50021
131::M::18::4::06520
132::M::25::17::99709
133::F::25::0::55071
134::M::25::0::66212
135::M::18::4::20006
136::M::18::2::21202
137::F::45::6::78758
138::M::18::20::22203
139::F::25::20::45409
140::F::35::1::55107
141::M::35::13::62035
142::M::25::7::10011
143::M::18::3::64043
144::M::25::17::29401
145::M::18::4::19081
146::F::35::20::10954
147::M::18::4::91360
148::M::50::17::57747
149::M::25::1::29205
150::M::35::7::98144
151::F::25::20::85013
152::M::18::4::48104
153::M::1::10::51537
154::M::50::20::94530
155::M::35::12::07470
156::F::45::7::14519
157::M::35::16::12866
158::M::56::7::28754
159::F::45::0::37922
160::M::35::7::13021
161::M::45::16::98107-2117
162::F::18::4::93117
163::M::18::4::85013
164::F::56::13::94566
165::M::18::16::98502
166::M::18::4::92802
167::F::25::11::10022
168::F::50::0::46970
169::M::25::7::55439
170::M::25::11::07002
171::F::50::17::55441
172::M::25::3::07661
173::M::25::0::45237
174::M::25::16::21203
175::F::25::2::95123
176::F::18::3::55016
177::M::50::1::54016
178::M::56::17::53705
179::M::25::0::02135
180::M::45::12::01603
181::M::18::17::33186
182::M::18::4::03052
183::F::45::1::55407
184::F::25::0::19001
185::M::45::0::14468
186::M::18::5::91767
187::F::45::1::94061
188::M::56::16::79930
189::M::18::0::60076
190::M::25::17::55125
191::M::1

In [7]:
#TODO: use dbutils.fs.head to inspect the file "ratings.dat"
ratings_filename = dbfs_dir + '/ratings.dat'
print(dbutils.fs.head(ratings_filename))

[Truncated to first 65536 bytes]
1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368
1::595::5::978824268
1::938::4::978301752
1::2398::4::978302281
1::2918::4::978302124
1::1035::5::978301753
1::2791::4::978302188
1::2687::3::978824268
1::2018::4::978301777
1::3105::5::978301713
1::2797::4::978302039
1::2321::3::978302205
1::720::3::978300760
1::1270::5::978300055
1::527::5::978824195
1::2340::3::978300103
1::48::5::978824351
1::1097::4::978301953
1::1721::4::978300055
1::1545::4::978824139
1::745::3::978824268
1::2294::4::978824291
1::3186::4::978300019
1::1566::4::978824330
1::588::4::978824268
1::1907::4::978824330
1::783::4::978824291
1::1836::5::978300172
1::1022::5::978300055
1::2762::4::978302091
1::150::5::978301777
1::1::5::978824268
1::1961::5::978301590
1::1962::4::978301753
1::2692::4::978301570
1::260::4::978300760
1::1028::5::978301777
1::1029::5::978302205
1::1207::4::978300719
1::2028::5::978301619
1::531::4::978302149
1::3114::4::978302174
1::608::4::978301398
1::1246::4::978302091
2::1357::5::978298709
2::3068::4::978299000
2::1537::4::978299620
2::647::3::978299351
2::2194::4::978299297
2::648::4::978299913
2::2268::5::978299297
2::2628::3::978300051
2::1103::3::978298905
2::2916::3::978299809
2::3468::5::978298542
2::1210::4::978298151
2::1792::3::978299941
2::1687::3::978300174
2::1213::2::978298458
2::3578::5::978298958
2::2881::3::978300002
2::3030::4::978298434
2::1217::3::978298151
2::3105::4::978298673
2::434::2::978300174
2::2126::3::978300123
2::3107::2::978300002
2::3108::3::978299712
2::3035::4::978298625
2::1253::3::978299120
2::1610::5::978299809
2::292::3::978300123
2::2236::5::978299220
2::3071::4::978299120
2::902::2::978298905
2::368::4::978300002
2::1259::5::978298841
2::3147::5::978298652
2::1544::4::978300174
2::1293::5::978298261
2::1188::4::978299620
2::3255::4::978299321
2::3256::2::978299839
2::3257::3::978300073
2::110::5::978298625
2::2278::3::978299889
2::2490::3::978299966
2::1834::4::978298813
2::3471::5::978298814
2::589::4::978299773
2::1690::3::978300051
2::3654::3::978298814
2::2852::3::978298958
2::1945::5::978298458
2::982::4::978299269
2::1873::4::978298542
2::2858::4::978298434
2::1225::5::978298391
2::2028::4::978299773
2::515::5::978298542
2::442::3::978300025
2::2312::3::978299046
2::265::4::978299026
2::1408::3::978299839
2::1084::3::978298813
2::3699::2::978299173
2::480::5::978299809
2::1442::4::978299297
2::2067::5::978298625
2::1265::3::978299712
2::1370::5::978299889
2::1193::5::978298413
2::1801::3::978300002
2::1372::3::978299941
2::2353::4::978299861
2::3334::4::978298958
2::2427::2::978299913
2::590::5::978299083
2::1196::5::978298730
2::1552::3::978299941
2::736::4::978300100
2::1198::4::978298124
2::593::5::978298517
2::2359::3::978299666
2::95::2::978300143
2::2717::3::978298196
2::2571::4::978299773
2::1917::3::978300174
2::2396::4::978299641
2::3735::3::978298814
2::1953::4::978298775
2::1597::3::978300025
2::3809::3::978299712
2::1954::5::978298841
2::1955::4::978299200
2::235::3::978299351
2::1124::5::978299418
2::1957::5::978298750
2::163::4::978299809
2::21::1::978299839
2::165::3::978300002
2::2321::3::978299666
2::1090::2::978298580
2::380::5::978299809
2::2501::5::978298600
2::349::4::978299839
2::457::4::978299773
2::1096::4::978299386
2::920::5::978298775
2::459::3::978300002
2::1527::4::978299839
2::3418::4::978299809
2::1385::3::978299966
2::3451::4::978298924
2::3095::4::978298517
2::780::3::978299966
2::498::3::978299418
2::2728::3::978298881
2::2002::5::978300100
2::1962::5::978298813
2::1784::5::978298841
2::2943::4::978298372
2::2006::3::978299861
2::318::5::978298413
2::1207::4::978298478
2::1968::2::978298881
2::3678::3::978299250
2::1244::3::978299143
2::356::5::978299686
2::1245::2::978299200
2::1246::5::978299418
2::3893::1::978299535
2::1247::5::978298652
3::3421::4::978

In [8]:
#TODO: use dbutils.fs.head to inspect the file "movies.dat"
movies_filename = dbfs_dir + '/movies.dat'
print(dbutils.fs.head(movies_filename))

[Truncated to first 65536 bytes]
1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller
11::American President, The (1995)::Comedy|Drama|Romance
12::Dracula: Dead and Loving It (1995)::Comedy|Horror
13::Balto (1995)::Animation|Children's
14::Nixon (1995)::Drama
15::Cutthroat Island (1995)::Action|Adventure|Romance
16::Casino (1995)::Drama|Thriller
17::Sense and Sensibility (1995)::Drama|Romance
18::Four Rooms (1995)::Thriller
19::Ace Ventura: When Nature Calls (1995)::Comedy
20::Money Train (1995)::Action
21::Get Shorty (1995)::Action|Comedy|Drama
22::Copycat (1995)::Crime|Drama|Thriller
23::Assassins (1995)::Thriller
24::Powder (1995)::Drama|Sci-Fi
25::Leaving Las Vegas (1995)::Drama|Romance
26::Othello (1995)::Drama
27::Now and Then (1995)::Drama
28::Persuasion (1995)::Romance
29::City of Lost Children, The (1995)::Adventure|Sci-Fi
30::Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)::Drama
31::Dangerous Minds (1995)::Drama
32::Twelve Monkeys (1995)::Drama|Sci-Fi
33::Wings of Courage (1995)::Adventure|Romance
34::Babe (1995)::Children's|Comedy|Drama
35::Carrington (1995)::Drama|Romance
36::Dead Man Walking (1995)::Drama
37::Across the Sea of Time (1995)::Documentary
38::It Takes Two (1995)::Comedy
39::Clueless (1995)::Comedy|Romance
40::Cry, the Beloved Country (1995)::Drama
41::Richard III (1995)::Drama|War
42::Dead Presidents (1995)::Action|Crime|Drama
43::Restoration (1995)::Drama
44::Mortal Kombat (1995)::Action|Adventure
45::To Die For (1995)::Comedy|Drama
46::How to Make an American Quilt (1995)::Drama|Romance
47::Seven (Se7en) (1995)::Crime|Thriller
48::Pocahontas (1995)::Animation|Children's|Musical|Romance
49::When Night Is Falling (1995)::Drama|Romance
50::Usual Suspects, The (1995)::Crime|Thriller
51::Guardian Angel (1994)::Action|Drama|Thriller
52::Mighty Aphrodite (1995)::Comedy
53::Lamerica (1994)::Drama
54::Big Green, The (1995)::Children's|Comedy
55::Georgia (1995)::Drama
56::Kids of the Round Table (1995)::Adventure|Children's|Fantasy
57::Home for the Holidays (1995)::Drama
58::Postino, Il (The Postman) (1994)::Drama|Romance
59::Confessional, The (Le Confessionnal) (1995)::Drama|Mystery
60::Indian in the Cupboard, The (1995)::Adventure|Children's|Fantasy
61::Eye for an Eye (1996)::Drama|Thriller
62::Mr. Holland's Opus (1995)::Drama
63::Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)::Comedy
64::Two if by Sea (1996)::Comedy|Romance
65::Bio-Dome (1996)::Comedy
66::Lawnmower Man 2: Beyond Cyberspace (1996)::Sci-Fi|Thriller
67::Two Bits (1995)::Drama
68::French Twist (Gazon maudit) (1995)::Comedy|Romance
69::Friday (1995)::Comedy
70::From Dusk Till Dawn (1996)::Action|Comedy|Crime|Horror|Thriller
71::Fair Game (1995)::Action
72::Kicking and Screaming (1995)::Comedy|Drama
73::Mis�rables, Les (1995)::Drama|Musical
74::Bed of Roses (1996)::Drama|Romance
75::Big Bully (1996)::Comedy|Drama
76::Screamers (1995)::Sci-Fi|Thriller
77::Nico Icon (1995)::Documentary
78::Crossing Guard, The (1995)::Drama
79::Juror, The (1996)::Drama|Thriller
80::White Balloon, The (Badkonake Sefid ) (1995)::Drama
81::Things to Do in Denver when You're Dead (1995)::Crime|Drama|Romance
82::Antonia's Line (Antonia) (1995)::Drama
83::Once Upon a Time... When We Were Colored (1995)::Drama
84::Last Summer in the Hamptons (1995)::Comedy|Drama
85::Angels and Insects (1995)::Drama|Romance
86::White Squall (1996)::Adventure|Drama
87::Dunston Checks In (1996)::Children's|Comedy
88::Black Sheep (1996)::Comedy
89::Nick of Time (1995)::Action|Thriller
90::Journey of August King, The (1995)::Drama
92::Mary Reilly (1996)::Drama|Thriller
93::Vampire in Brookly

### Ejercicio 1(b):

Cargar los diferentes ficheros (usuarios, películas y calificaciones) en tres RDDs para su posterior procesado.

In [10]:
# TODO: Load the users data and print the first five lines.
rawUsersTextRdd = sc.textFile(users_filename)
print rawUsersTextRdd.count()
print rawUsersTextRdd.take(5)

6040
[u'1::F::1::10::48067', u'2::M::56::16::70072', u'3::M::25::15::55117', u'4::M::45::7::02460', u'5::M::25::20::55455']

In [11]:
# TODO: Load the ratings data and print the first five lines.
rawRatingsTextRdd = sc.textFile(ratings_filename)
print rawRatingsTextRdd.take(5)

[u'1::1193::5::978300760', u'1::661::3::978302109', u'1::914::3::978301968', u'1::3408::4::978300275', u'1::2355::5::978824291']

In [12]:
# TODO: Load the movies data and print the first five lines.
rawMoviesTextRdd = sc.textFile(movies_filename)
print rawMoviesTextRdd.take(5)

[u"1::Toy Story (1995)::Animation|Children's|Comedy", u"2::Jumanji (1995)::Adventure|Children's|Fantasy", u'3::Grumpier Old Men (1995)::Comedy|Romance', u'4::Waiting to Exhale (1995)::Comedy|Drama', u'5::Father of the Bride Part II (1995)::Comedy']

### Ejercicio 1(c): 

Como has observado inspecionando los ficheros, el fichero de usuarios contiene algunas líneas con más de un zip-code. Para simplicidad, eliminaremos estas líneas del fichero. Para eso crearemos una función que elimine todas las líneas que contenga algún caracter que no sea '0123456789:MF'.

In [14]:
#TODO: Complete the function validLine
import re
def validLine(line):
    """Verifies if a line is valid to be converted to a dataframe.
    Args:
        line (str): A string.

    Returns:
        boolean: True if valid, False otherwise.
    """
    return not re.compile('[^0-9::MF\\s]').search(line)

In [15]:
# Load in the testing code and check to see if your answer is correct
# If incorrect it will report back '1 test failed' for each failed test
# Make sure to rerun any cell you change before trying the test again
from databricks_test_helper import Test
# TEST validLine (1b)
Test.assertEquals(validLine('1::F::1::10::48067'), True, 'incorrect result: validLine does not recognize correct lines')
Test.assertEquals(validLine('161::M::45::16::98107-2117'), False, 'incorrect result: validLine does not remove the incorrect lines')

1 test passed.
1 test passed.

### Ejercicio 1(d)

Como en anteriores ocasiones, crea un esquema a medida para cada uno de los ficheros.

In [17]:
#TODO: Fill in the user schema.
from pyspark.sql.types import *

# Custom Schema for users
userSchema = StructType([ \
    StructField('UserID', IntegerType(), True), \
    StructField('Gender' , IntegerType(), True), \
    StructField('Age', IntegerType(), True), \
    StructField('Occupation', IntegerType(), True), \
    StructField('Zip-code', LongType(), True)])

In [18]:
# Custom Schema for ratings
ratingsSchema = StructType([ \
    StructField('UserID', IntegerType(), True), \
    StructField('MovieID', IntegerType(), True), \
    StructField('Rating' , IntegerType(), True), \
    StructField('Timestamp', LongType(), True)])

In [19]:
# Custom Schema for movies
moviesSchema = StructType([ \
    StructField('MovieID', IntegerType(), True), \
    StructField('Title',  StringType(), True), \
    StructField('Genres',  StringType(), True)])

### Ejercicio 1(e)

Elimina las líneas no válidas contenidas en `rawUsersTextRdd`.

In [21]:
#TODO: filter invalid lines
filteredUsersTextRdd = rawUsersTextRdd.filter(validLine)

In [22]:
# TEST filter validLines (1d)
counter = filteredUsersTextRdd.count()
Test.assertEquals(counter, 5974, 'incorrect result: lines are incorrectly filtered')

1 test passed.

### Ejercicio 1(f)

Para poder usar fácilmente diferentes algoritmos de clustering es conveniente que todos los atributos sean numéricos, para esto convertiremos el atributo gender de los usuarios en 1 si es hombre ('M') o 0 si es mujer ('F'), cambia en el esquema que has creado anteriormente el atributo gender a `IntegerType()` si lo tenías como `char` o `string`.

Luego transforma el RDD en un DataFrame usando la función [toDF()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) indicándole que schema debe usar como un parámetro de la función (`schema= customSchema`).

Para realizar esta transformación a DataFrame, primero hay que convertir cada línea en una lista de enteros, ya que así se indica en nuestro esquema. Para esto usaremos las funciones de Python [split()](https://docs.python.org/2/library/stdtypes.html#str.split) y [int()](https://docs.python.org/2/library/functions.html#int). Puedes combinar estas dos transformaciones junto con la conversión del atributo `gender` en la misma función lambda o hacer una función con nombre. Luego usa una función `map` para aplicar los cambios a todas las lineas del RDD de usuarios.

Finalmente, comprueba que los datos mostrados por el comando `display(usersDF)` son correctos.

In [24]:
#TODO: transform the Gender type and create a DataFrame
def convert_gender(line):
  return (line.replace("M", "1")).replace("F", "0")

def parse_users(line):
  """
  Parses an user record in a format UserID::Gender::Age::Occupation::Zip-code.
  """
  fields = line.strip().split("::")
  return int(fields[0]), int(fields[1]), int(fields[2]), int(fields[3]), long(fields[4])

integerGendersUsersTextRdd = filteredUsersTextRdd.map(convert_gender).map(parse_users)
integerGendersUsersTextRdd.take(10)
usersDF = integerGendersUsersTextRdd.toDF(schema = userSchema)

display(usersDF)

UserID,Gender,Age,Occupation,Zip-code
1,0,1,10,48067
2,1,56,16,70072
3,1,25,15,55117
4,1,45,7,2460
5,1,25,20,55455
6,0,50,9,55117
7,1,35,1,6810
8,1,25,12,11413
9,1,25,17,61614
10,0,35,1,95370


## Ejercicio 1(g)

Ahora aplica las mismas transformaciones, excepto la del atributo gender, al RDD de ratings.

In [26]:
def parse_ratings(line):
  """
  Parses a rating record in a format UserID::MovieID::Rating::Timestamp.
  """
  fields = line.strip().split("::")
  return int(fields[0]), int(fields[1]), int(fields[2]), long(fields[3])

#TODO: Create a DataFrame for ratings RDD
ratingsDF = rawRatingsTextRdd.map(parse_ratings).toDF(schema = ratingsSchema)

display(ratingsDF)

UserID,MovieID,Rating,Timestamp
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368


## Parte 2: Ingeniería de características

La ingeniería de características es el proceso de utilizar el conocimiento del dominio de los datos para crear las características que hacen que los algoritmos de machine learning trabajen de forma correcta. Esto es fundamental en la aplicación del machine learning a datos del mundo real, y en general es difícil y costosa. La ingeniería de características es un tema informal, pero se considera esencial en el machine learning aplicado.

### Ejercicio 2(a)

Para esta PAC calcularemos unas cuantas características muy sencillas como el número de películas vistas por cada usuario, la media de sus calificaciones y su varianza.

Para obtener estas características usaremos la función [groupBy()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) de los DataFrames de Spark, así como las funciones de agregación que nos proporciona SparkQL (https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#module-pyspark.sql.functions).

Por simplicidad usaremos la función [alias()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html#pyspark.sql.SQLContext) para poner los siguientes nombres a las columnas: NumMovies, AvgRating y VarRating.

In [29]:
#TODO: Compute aggregated values by user using groupBy and SQL functions
import pyspark.sql.functions as func

aggRatingsDF = ratingsDF.groupBy("UserID").agg(func.count(ratingsDF.MovieID).alias("NumMovies"), func.avg(ratingsDF.Rating).alias("AvgRating"), func.variance(ratingsDF.Rating).alias("VarRating"))

display(aggRatingsDF)

UserID,NumMovies,AvgRating,VarRating
148,624,3.733974358974359,0.9564040828085761
463,123,3.0,1.5081967213114758
471,105,3.6285714285714286,1.543406593406593
496,119,4.294117647058823,0.9042871385842477
833,21,4.0476190476190474,0.9476190476190473
1088,1176,3.3375850340136055,1.12253509914604
1238,45,3.7111111111111112,0.8919191919191914
1342,92,2.9782608695652173,1.0544672718585761
1580,37,3.5405405405405403,0.644144144144144
1591,314,4.340764331210191,0.6598665065830978


In [30]:
# TEST compute aggregated features (2a)
counter = aggRatingsDF.count()
result = aggRatingsDF.where(aggRatingsDF.UserID == 31).first()
Test.assertEquals(counter, 6040, 'incorrect result: aggregation is incorreclty done')
Test.assertEquals(result["NumMovies"], 119, 'incorrect result: NumMovies is incorrect')
Test.assertEquals(result["AvgRating"], 3.73109243697479, 'incorrect result: AvgRating is incorrect')
Test.assertEquals(result["VarRating"], 1.0457199829084174, 'incorrect result: NumMovies is incorrect')
print result

1 test passed.
1 test passed.
1 test passed.
1 test passed.
Row(UserID=31, NumMovies=119, AvgRating=3.73109243697479, VarRating=1.0457199829084174)

###Ejercicio 2(b)

Ahora juntaremos las características que hemos extraido del fichero de ratings con el fichero de usuarios. Como no queremos tener que preocuparnos por los valores nulos en las características, solo nos quedaremos con los usuarios que hayan calificado alguna película. Para realizar esto, usaremos un inner join. Encontraremos los detalles de esta función aquí [join()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.sql.html).

In [32]:
#TODO: Compute an inner join between usersDF and aggRatingsDF
joinedDF = aggRatingsDF.join(usersDF, ["UserID"])#aggRatingsDF.UserID == usersDF.UserID)
display(joinedDF)

UserID,NumMovies,AvgRating,VarRating,Gender,Age,Occupation,Zip-code
148,624,3.733974358974359,0.9564040828085761,1,50,17,57747
463,123,3.0,1.5081967213114758,1,25,7,55105
471,105,3.6285714285714286,1.543406593406593,1,35,7,8904
496,119,4.294117647058823,0.9042871385842477,1,18,4,55455
833,21,4.0476190476190474,0.9476190476190473,1,35,7,46825
1088,1176,3.3375850340136055,1.12253509914604,0,1,10,98103
1238,45,3.7111111111111112,0.8919191919191914,0,35,20,11215
1342,92,2.9782608695652173,1.0544672718585761,1,35,0,94560
1580,37,3.5405405405405403,0.644144144144144,0,18,4,76201
1591,314,4.340764331210191,0.6598665065830978,1,50,7,26501


In [33]:
# TEST join (2b)
counter = joinedDF.count()
Test.assertEquals(counter, 5974, 'incorrect result: join is incorreclty done')

1 test passed.

### Ejercicio 2(c)

Ahora vamos a guardar los datos generados para poder reutilizarlos en un futuro sin tener que re-ejecutar todo el notebook, para esto ejecutaremos el siguiente código.

In [35]:
#TODO: Read, understand and execute the cell
sqlContext.sql("DROP TABLE IF EXISTS joinedDF")
dbutils.fs.rm("dbfs:/user/hive/warehouse/joinedDF", True)
sqlContext.registerDataFrameAsTable(joinedDF, "joinedDF")

## Parte 3: Clustering

Un algoritmo de agrupamiento (en inglés, clustering) es un procedimiento de agrupación de una serie de vectores de acuerdo con un criterio. Esos criterios son por lo general distancia o similitud. La cercanía se define en términos de una determinada función de distancia, como la de Euclides, aunque existen otras mas robustas o que permiten extenderla a variables discretas.

Existen dos grandes familias de clustering:

* *Agrupamiento jerárquico*, que puede ser aglomerativo o divisivo.
* *Agrupamiento no jerárquico*, en los que el número de grupos se determina de antemano y las observaciones se van asignando a los grupos en función de su cercanía. En esta familia existen una gran cantidad de métodos, en esta PAC usaremos el método de k-means (k-medias).

### k-means

K-means es un método de clustering, que tiene como objetivo la partición de un conjunto de _n_ observaciones en _k_ grupos en el que cada observación pertenece al grupo cuyo valor medio es más cercano. Una de las ventajas de este método es que la agrupación del conjunto de datos puede ilustrarse en una partición del espacio de datos en [celdas de Voronoi](https://es.wikipedia.org/wiki/Pol%C3%ADgonos_de_Thiessen#Diagramas_de_Voron.C3.B3i_en_el_plano_euclidiano_.7F.27.22.60UNIQ--postMath-00000001-QINU.60.22.27.7F).

El problema es computacionalmente difícil (NP-hard). Sin embargo, hay heurísticas muy eficientes que se emplean comunmente y que convergen rápidamente a un óptimo local.

El algoritmo de k-means más común utiliza una técnica de refinamiento iterativo, tal y como se describe a continuación:

Dado un conjunto inicial de _k_ centroides $$ m_1^{(1)},...,m_k^{(1)} $$ el algoritmo continua alternando entre estos dos pasos:

* *Paso de asignación:* Asigna cada observación al grupo con la media más cercana (es decir, la partición de las observaciones de acuerdo con el diagrama de Voronoi generado por los centroides).

$$ S_{i}^{(t)} = \\{ x_p: || x_p - m_i^{(t)} || \leq || x_p - m_j^{(t)} || \forall 1 \leq j \leq k \\} $$

* *Paso de actualización:* Calcular los nuevos centroides como el centroide de las observaciones en el grupo.
$$ m_i^{(t+1)} = \frac{1}{|S_i^{(t)}|} \sum^{x_j \in S_i^{(t)}} x_j$$

El algoritmo se considera que ha convergido cuando las asignaciones ya no cambian. Los centroides suelen iniciarse de forma aleatoria.

El siguiente paso es preparar los datos para aplicar el k-means. Dado que todo el dataset es numérico y consistente, esta será una tarea sencilla y directa.

El objetivo es utilizar el método de clustering para determinar _k_ usuarios estándar (promedio) que representen la totalidad de los usuarios que tenemos en nuestro dataset. El primer paso en la construcción de nuestro modelo de clustering es convertir las características que hemos calculado en nuestro DataFrame a un vector de características utilizando el método [pyspark.ml.feature.VectorAssembler()](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.feature.VectorAssembler).

El VectorAssembler es una transformación que combina una lista dada de columnas en una único vector. Esta transformación es muy útil cuando queremos combinar características en crudo de los datos con otras generadas al aplicar diferentes funciones sobre los datos en un único vector de características. Para integrar en un único vector toda esta información antes de ejecutar un algoritmo de aprendizaje automático, el VectorAssembler toma una lista con los nombres de las columnas de entrada (lista de strings) y el nombre de la columna de salida (string).

### Ejercicio 3(a)

- leer la documentación y los ejemplos de uso de [VectorAssembler](https://spark.apache.org/docs/1.6.2/ml-features.html#vectorassembler)
- Convertir la tabla SQL `joinedDF` en un `dataframe` llamado datasetDF usando la función table del sqlContext
- Establecer las columnas de entrada del VectorAssember: `['Age','NumMovies','AvgRating','VarRating']`
- Establecer la columnas de salida como `"features"`

In [38]:
#TODO: Replace <FILL_IN> with the appropriate code
from pyspark.ml.feature import VectorAssembler

datasetDF = sqlContext.sql("SELECT * FROM joinedDF")
input_cols = ['Age','NumMovies','AvgRating','VarRating']

vectorizer = VectorAssembler()
vectorizer.setInputCols(input_cols)
vectorizer.setOutputCol("features")

Out[ 27 ]: VectorAssembler_41f283d2def392244b0f

### Ejercicio 3(b)

Guardar en la memoria cache el dataframe `datasetDF` y renombrarlo a clusteringDF

In [40]:
#TODO: Let's cache the dataset for performance
clusteringDF = datasetDF.cache()

### Ejercicio 3(c)

- Leer la documentación y los ejemplos de [k-means](https://spark.apache.org/docs/1.6.2/ml-clustering.html#k-means)
- Ejecutar la siguiente celda

In [42]:
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

kmeans = KMeans()
print(kmeans.explainParams())

featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -7649703878154674547)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)

La siguiente celda está basada en [Spark 2.0.1 ML Pipeline API for clustering](https://spark.apache.org/docs/2.0.1/mllib-clustering.html#k-means).

El primer paso es establecer los valores de los parámetros:
- Definir el número de clusters (k) como 10
- Definir el número máximo de iteraciones a 25
- Definir el random seed a 1

Ahora, crearemos el [ML Pipeline](https://spark.apache.org/docs/1.6.2/api/python/pyspark.ml.html#pyspark.ml.Pipeline) (flujo de ejecución) y estableceremos las fases del pipeline como vectorizar y posteriormente aplicar el algoritmo de clustering que hemos definido.

Finalmente, crearemos el modelo entrenándolo con el DataFrame `clusteringDF`.

### Ejercicio 3(d)

- Leer la documentación [k-means](https://spark.apache.org/docs/2.0.1/mllib-clustering.html#k-means).
- Completa y ejecuta la siguiente celda, asegúrate de entender que es lo que sucede.

In [44]:
## TODO: Replace <FILL_IN> with the appropriate code
# Now we set the parameters for the method
def generate_kmeans(myDF):
  kmeans = KMeans().setK(10).setSeed(1).setMaxIter(25)
  
  clusteringPipeline = Pipeline()
  # We will use the new spark.ml pipeline API. If you have worked with scikit-learn this will be very familiar.
  clusteringPipeline.setStages([vectorizer, kmeans])

  # Let's train on the entire dataset to see what we get
  return clusteringPipeline.fit(myDF)

### Ejercicio 3(e)

Ejecutar la siguiente celda y observar los centroides que has obtenido. Luego responde a las siguientes preguntas:

- ¿Crees que los centrodides son significativos?
- ¿Qué crees que ha sucedido?

In [46]:
# Shows the result.
def print_centroids(model):
  centers = model.stages[1].clusterCenters()
  print("Centroids: ")
  for center in centers:
      print(center)

clusteringModel = generate_kmeans(clusteringDF)
print_centroids(clusteringModel)

Centroids: 
[ 31.78483607 40.53852459 3.77152261 1.0874308 ]
[ 29.79411765 679.17647059 3.38708121 1.10186268]
[ 28.6993865 385.24233129 3.56114818 1.04613826]
[ 30.49588719 177.95182139 3.6853088 1.03354499]
[ 28.98173516 514.49315068 3.43279338 1.09492468]
[ 2.95200000e+01 1.26076000e+03 3.29501967e+00 1.14631626e+00]
[ 27.93478261 882.26086957 3.36729196 1.09679141]
[ 28.89836661 275.43375681 3.62676643 1.04342411]
[ 30.13899324 101.91059354 3.75688228 1.03437426]
[ 3.66666667e+01 1.96900000e+03 3.74745298e+00 9.92322694e-01]

## Parte 4: Validación de los resultados

Ahora estudiaremos como se comportan nuestras predicciones con este modelo.

Para realizar esta validación, utilizaremos el [Error cuadrático medio](https://en.wikipedia.org/wiki/Root-mean-square_deviation) (RMSE).

Recordar que el RSME se define como: \\( RMSE = \sqrt{\frac{\sum_{i = 1}^{n} (x_i - y_i)^2}{n}}\\) donde \\(y_i\\) es el valor observado \\(x_i\\) es el valor predicho

RMSE es una medida muy habitual en los métodos de clustering para calcular las diferencias entre los valores predichos (centroides) y los elementos que componen ese cluster. Cuanto menor sea el RMSE, mejor será nuestro método de clustering.

### Ejercicio 4(a)

El modelo de `k-means` de Spark proporciona un método para calcular la suma de las distancias al cuadrado de todos los elementos de un DataFrame a su centroide más cercano. Esta medida se conoce como SSE (_Sum of Square Errors_). A partir de este valor podemos a calcular el RMSE.

In [49]:
## TODO: Replace <FILL_IN> with the appropriate code
import math

def calculateRMSE(clusteringModel, df):
  kmeansModel = clusteringModel.stages[1]
  transformedDF = clusteringModel.stages[0].transform(df)

  #Obtain the SSE
  SSE = kmeansModel.computeCost(transformedDF)

  #obtain the number of elements of the DataFrame clusteringDF
  n = transformedDF.count()

  #compute RMSE
  RMSE = math.sqrt(SSE/n)
  print ("Error cuadratico medio: " + str(RMSE))
  
  return n, RMSE, transformedDF

In [50]:
n, RMSE, transformedDF = calculateRMSE(clusteringModel, clusteringDF)
#TEST RMSE 4(a)
Test.assertEquals(n, 5974, 'incorrect result: number of elements is wrong')
Test.assertEquals(round(RMSE,3), 29.437, 'incorrect result: RMSE error')

Error cuadratico medio: 29.4368319884
1 test passed.
1 test passed.

### Ejercicio 4(b)

Inspecciona visualmente si los centroides predichos de algunos de los elementos de clusteringDF están cerca de los valores reales y si tienen sentido, luego responde las siguientes preguntas:

- ¿Como afecta el hecho de no haber normalizado los datos? 
- ¿Hay algún atributo más importante que los demás? En caso que la respuesta sea afirmativa, ¿Cual?

Ejecuta la siguiente celda para ver en que cluster se han asignado cada uno de los registros del DataFrame. Los identificadores de cluster empiezan por 0.

In [52]:
display(kmeansModel.transform(transformedDF).select(['Age','NumMovies','AvgRating','VarRating','prediction']))

Age,NumMovies,AvgRating,VarRating,prediction
50,624,3.733974358974359,0.9564040828085761,1
25,123,3.0,1.5081967213114758,8
35,105,3.6285714285714286,1.543406593406593,8
18,119,4.294117647058823,0.9042871385842477,8
35,21,4.0476190476190474,0.9476190476190473,0
1,1176,3.3375850340136055,1.12253509914604,5
35,45,3.7111111111111112,0.8919191919191914,0
35,92,2.9782608695652173,1.0544672718585761,8
18,37,3.5405405405405403,0.644144144144144,0
50,314,4.340764331210191,0.6598665065830978,7


## Parte 5: Normalización de los datos

Una buena practica cuando usamos métodos de machine learning basados en distancias es asegurarnos que todos los atributos están en la misma escala. Esto no sucede en nuestro dataset ya que cada atributo tiene un rango diferente.

Prueba las siguientes opciones de normalización y re-ejecuta el notebook con los datos normalizados:

* **normalización:** Convierte todos los atributos al rango [0,1] usando la siguiente fórmula
$$ x' = \frac{x - min}{max-min} $$

* **estandarización (z-scores):** Convierte todos los atributos en una distribución normal con media = 0 y varianza = 1 usando la siguiente fórmula
$$ z = \frac{x - \mu}{\sigma}$$

- ¿Tienen ahora los centroides más sentido?
- ¿Has conseguido reducir error? ¿Esto implica que ahora los centroides están mejor calculados?

### Respuesta

Voy a empezar por la creación del codigo que hace falta para contestar las preguntas y en la secuencia haré los comentarios que responden a las 2 preguntas anteriormente hechas:

In [54]:
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col

input_normalized = []
input_standardized = []

def normalizing_standardizing(df):
  print df.count()
  # All info: UserID  NumMovies AvgRating VarRating Gender  Age Occupation  Zip-code
  for col_name in input_cols:  # Being used ['Age','NumMovies','AvgRating','VarRating']
    norm = col_name + "_nm"
    stan = col_name + "_st"
    input_normalized.append(norm)
    input_standardized.append(stan)
    
    df_stats = df.select(_mean(col(col_name)).alias('mean'), _stddev(col(col_name)).alias('std')).collect()
    
    # Recovering variables used to calculate
    max = df.agg({col_name: "max"}).collect()[0][0]
    min = df.agg({col_name: "min"}).collect()[0][0]
    mean2 = df.agg({col_name: "mean"}).collect()[0][0]
    mean = df_stats[0]['mean']
    std = df_stats[0]['std']

    df = df.withColumn(norm, ((df[col_name])-min)/(max-min))
    df = df.withColumn(stan, ((df[col_name])-mean)/std)
    
  return input_normalized, input_standardized, df


input_normalized,input_standardized, dfFullyNorm = normalizing_standardizing(clusteringDF)

display(dfFullyNorm)

UserID,NumMovies,AvgRating,VarRating,Gender,Age,Occupation,Zip-code,Age_nm,Age_st,NumMovies_nm,NumMovies_st,AvgRating_nm,AvgRating_st,VarRating_nm,VarRating_st
148,624,3.733974358974359,0.9564040828085761,1,50,17,57747,0.8909090909090909,1.508921238170985,0.2632955536181343,2.3917339115218486,0.6886727771362587,0.07507740093866255,0.27281810403185486,-0.2466457935157123
463,123,3.0,1.5081967213114758,1,25,7,55105,0.43636363636363634,-0.4341163618086379,0.044899738448125545,-0.22006399226117823,0.502742494226328,-1.6348264927079756,0.4327925005947614,1.0353127434692917
471,105,3.6285714285714286,1.543406593406593,1,35,7,8904,0.6181818181818182,0.34309867818321127,0.037053182214472534,-0.31390104269649655,0.6619721214120753,-0.1704746117359827,0.44300046215212435,1.117114480152389
496,119,4.294117647058823,0.9042871385842477,1,18,4,55455,0.3090909090909091,-0.9781668898029323,0.043156059285091544,-0.2409166701356934,0.8305681972558077,1.380015615175539,0.25770848472701924,-0.36772707823748746
833,21,4.0476190476190474,0.9476190476190473,1,35,7,46825,0.6181818181818182,0.34309867818321127,4.3591979075850045E-4,-0.7518072780613154,0.7681252062025734,0.8057599755786788,0.2702711677279531,-0.26705572665560695
1088,1176,3.3375850340136055,1.12253509914604,0,1,10,98103,0.0,-2.299432457789076,0.5039232781168265,5.269403458204944,0.5882594882641279,-0.8483712752487206,0.3209824093061604,0.13931998382708916
1238,45,3.7111111111111112,0.8919191919191914,0,35,20,11215,0.6181818181818182,0.34309867818321127,0.010897994768962511,-0.6266912108142243,0.6828810623556583,0.021814019098723797,0.254122799332896,-0.39646105130255355
1342,92,2.9782608695652173,1.0544672718585761,1,35,0,94560,0.6181818181818182,0.34309867818321127,0.03138622493461203,-0.38167224578867087,0.49723554071693943,-1.6854710735716316,0.3012483482914936,-0.018819371786526356
1580,37,3.5405405405405403,0.644144144144144,0,18,4,76201,0.3090909090909091,-0.9781668898029323,0.0074106364428945075,-0.6683965665632546,0.6396721490543661,-0.3755558333954514,0.18228845254149667,-0.9721072552984775
1591,314,4.340764331210191,0.6598665065830978,1,50,7,26501,0.8909090909090909,1.508921238170985,0.12816041848299914,0.7756513762469218,0.8423847288212883,1.4886860964817048,0.18684664210078292,-0.9355800975515494


In [55]:
clusteringDFStan = dfFullyNorm.select("UserID", col("Age_st").alias("Age"), col("NumMovies_st").alias("NumMovies"), col("AvgRating_st").alias("AvgRating"), col("VarRating_st").alias("VarRating"), "Gender", "Occupation", "Zip-code")

clusteringModelStan = generate_kmeans(clusteringDFStan)
print_centroids(clusteringModelStan)
nS, RMSE_Stan, transformedDFStan = calculateRMSE(clusteringModelStan, clusteringDFStan)
Test.assertEquals(nS, 5974, 'incorrect result: number of elements is wrong')

Centroids: 
[-0.26292913 -0.34113468 1.11051163 -0.82503223]
[-0.20244649 3.64597737 -0.93754744 0.19329453]
[-0.4905807 -0.30418762 -1.215634 2.42070609]
[-0.58780295 0.06015123 -1.53229984 0.46996368]
[-0.12794074 1.41660173 -0.21127305 -0.19443801]
[ 1.480287 -0.3171204 -0.40141741 0.25465647]
[-0.34438145 -0.41275507 0.11862072 0.85456962]
[ 1.51038132 -0.3406675 0.91019368 -0.83714537]
[-1.36525641 -0.32097751 0.20564408 -0.15621344]
[-0.10344345 -0.31197002 -0.08810119 -0.43836826]
Error cuadratico medio: 1.10653684489
1 test passed.

In [56]:
clusteringDFNorm = dfFullyNorm.select("UserID", col("Age_nm").alias("Age"), col("NumMovies_nm").alias("NumMovies"), col("AvgRating_nm").alias("AvgRating"), col("VarRating_nm").alias("VarRating"), "Gender", "Occupation", "Zip-code")

clusteringModelNorm = generate_kmeans(clusteringDFNorm)
print_centroids(clusteringModelNorm)
nN, RMSE_Norm, transformedDFNorm = calculateRMSE(clusteringModelNorm, clusteringDFNorm)
Test.assertEquals(nN, 5974, 'incorrect result: number of elements is wrong')

Centroids: 
[ 0.61818182 0.04983738 0.7095861 0.25973942]
[ 0. 0.04523071 0.66598182 0.35516357]
[ 0.44914396 0.28929464 0.58171243 0.31239196]
[ 0.70624506 0.04605777 0.57131685 0.52041482]
[ 0.88354545 0.05695801 0.63816155 0.3187023 ]
[ 0.37321538 0.04833005 0.56337269 0.50936664]
[ 0.30909091 0.04984529 0.68704931 0.28678061]
[ 0.89065083 0.04213251 0.77712087 0.19574953]
[ 0.42987013 0.04113012 0.79402824 0.18785611]
[ 0.43636364 0.04951723 0.66872913 0.31007638]
Error cuadratico medio: 0.140918273875
1 test passed.

- **¿Tienen ahora los centroides más sentido?**

Sí, los centroides están mejores calculados y tiene mas sentido las informaciones que notamos en los *reports* anteriores. Esto pasa porque la normalización de datos (paso de preparación de datos requerido para muchos algoritmos de aprendizaje automático) es importante porque algoritmos son sensibles a los valores relativos de los atributos *feature*. La normalización de datos es el proceso de llevar todos los valores de atributo dentro de un rango deseado. A menos que los datos estén normalizados, estos algoritmos no se comportan correctamente.
<p></p>


- **¿Has conseguido reducir error? ¿Esto implica que ahora los centroides están mejor calculados?**

Sí, hay reducciones significativas como podemos notar en las sentencias *"Error cuadratico medio"* arriba en la magnitud de más de 20 veces mejor.